In [3]:
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [6]:
# Initialize lists to store images and labels
train_data = []
train_labels = []

# Path to the training folder
train_path = 'images\\train'

# Define the emotion label to integer mapping
emotion_mapping = {
    'angry': 0,
    'happy': 1,
    'neutral': 2,
    'sad': 3,
    'surprise': 4
}

# Iterate through each emotion folder in the training directory
for emotion in os.listdir(train_path):
    emotion_path = os.path.join(train_path, emotion)
    for img_file in os.listdir(emotion_path):
        img_path = os.path.join(emotion_path, img_file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (48, 48))  # Resize images to 48x48 pixels
        train_data.append(img)
        train_labels.append(emotion_mapping[emotion])  # Map the emotion label to an integer

# Convert to NumPy arrays and normalize
train_data = np.array(train_data)
train_data = train_data.astype('float32') / 255.0

# Reshape to add the channel dimension
train_data = np.expand_dims(train_data, -1)  # Shape: (samples, 48, 48, 1)

# Convert labels to categorical (one-hot encoding)
train_labels = np.array(train_labels)
train_labels = to_categorical(train_labels, num_classes=5)

# Similarly, load and preprocess the test dataset
test_data = []
test_labels = []
test_path = 'images\\validation'

for emotion in os.listdir(test_path):
    emotion_path = os.path.join(test_path, emotion)
    for img_file in os.listdir(emotion_path):
        img_path = os.path.join(emotion_path, img_file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (48, 48))
        test_data.append(img)
        test_labels.append(emotion_mapping[emotion])  # Map the emotion label to an integer

test_data = np.array(test_data)
test_data = test_data.astype('float32') / 255.0

# Reshape to add the channel dimension
test_data = np.expand_dims(test_data, -1)  # Shape: (samples, 48, 48, 1)

# Convert labels to categorical (one-hot encoding)
test_labels = np.array(test_labels)
test_labels = to_categorical(test_labels, num_classes=5)


In [7]:
# Initialize a sequential model
model = Sequential()

# First convolutional layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Second convolutional layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Third convolutional layer
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Fully connected layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Output layer with 7 neurons (for 7 emotion classes)
model.add(Dense(5, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Display the model's architecture
model.summary()

C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 355,589 (1.36 MB)

 Trainable params: 355,589 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation to improve generalization
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

datagen.fit(train_data)

# Train the model
history = model.fit(datagen.flow(train_data, train_labels, batch_size=64),
                    epochs=25,
                    validation_data=(test_data, test_labels))


Epoch 1/25


C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


380/380 ━━━━━━━━━━━━━━━━━━━━ 20s 47ms/step - accuracy: 0.2866 - loss: 1.5781 - val_accuracy: 0.3493 - val_loss: 1.4753
Epoch 2/25
380/380 ━━━━━━━━━━━━━━━━━━━━ 19s 49ms/step - accuracy: 0.3410 - loss: 1.4903 - val_accuracy: 0.4413 - val_loss: 1.3637
Epoch 3/25
380/380 ━━━━━━━━━━━━━━━━━━━━ 51s 134ms/step - accuracy: 0.3892 - loss: 1.4193 - val_accuracy: 0.5147 - val_loss: 1.2116
Epoch 4/25
380/380 ━━━━━━━━━━━━━━━━━━━━ 105s 276ms/step - accuracy: 0.4448 - loss: 1.3269 - val_accuracy: 0.5533 - val_loss: 1.1190
Epoch 5/25
380/380 ━━━━━━━━━━━━━━━━━━━━ 17s 45ms/step - accuracy: 0.4781 - loss: 1.2622 - val_accuracy: 0.5617 - val_loss: 1.0986
Epoch 6/25
380/380 ━━━━━━━━━━━━━━━━━━━━ 72s 189ms/step - accuracy: 0.5024 - loss: 1.2169 - val_accuracy: 0.5796 - val_loss: 1.0236
Epoch 7/25
380/380 ━━━━━━━━━━━━━━━━━━━━ 30s 79ms/step - accuracy: 0.5252 - loss: 1.1738 - val_accuracy: 0.6050 - val_loss: 0.9818
Epoch 8/25
380/380 ━━━━━━━━━━━━━━━━━━━━ 55s 144ms/step - accuracy: 0.5395 - loss: 1.1399 - val_ac

In [13]:
from tensorflow.keras.models import save_model
# Save the model
model.save('emotion_detection_model.h5')


In [22]:
import cv2
import numpy as np
import pygame
from tensorflow.keras.models import load_model

# Initialize Pygame
pygame.init()

# Load the saved model
model = load_model('emotion_detection_model.h5')

# Load the Haar cascade for frontal face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load and resize emoji images for different emotions
emoji_size = (159, 150)  # Desired emoji size
emojis = {
    'Angry': pygame.transform.scale(pygame.image.load('angry.png'), emoji_size),
    'Happy': pygame.transform.scale(pygame.image.load('happy-face.png'), emoji_size),
    'Neutral': pygame.transform.scale(pygame.image.load('neutral.png'), emoji_size),
    'Sad': pygame.transform.scale(pygame.image.load('sad.png'), emoji_size),
    'Surprise': pygame.transform.scale(pygame.image.load('surprised.png'), emoji_size),
}

# Initialize webcam feed
cap = cv2.VideoCapture(0)

# Set up Pygame display window
screen = pygame.display.set_mode((640, 480))

def detect_face_and_predict_emotion(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    emotion_label = None

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48))
        roi_gray = roi_gray.astype('float32') / 255.0
        roi_gray = np.expand_dims(roi_gray, axis=0)
        roi_gray = np.expand_dims(roi_gray, axis=-1)

        prediction = model.predict(roi_gray)
        emotion = np.argmax(prediction)
        emotion_label = ['Angry', 'Happy', 'Neutral', 'Sad', 'Surprise'][emotion]

        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, emotion_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

    return frame, emotion_label

def display_emoji(screen, emotion_label):
    if emotion_label and emotion_label in emojis:
        emoji = emojis[emotion_label]
        emoji_rect = emoji.get_rect()

        # Set position to upper right corner
        screen_width, screen_height = screen.get_size()
        x = screen_width - emoji_rect.width - 10  # 10 pixels from the right edge
        y = 10  # 10 pixels from the top edge

        screen.blit(emoji, (x, y))
        pygame.display.update()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame, emotion_label = detect_face_and_predict_emotion(frame)
    
    # Convert OpenCV frame to Pygame surface without rotation or flipping
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_surface = pygame.surfarray.make_surface(np.transpose(frame, (1, 0, 2)))  # Correct orientation

    # Display the frame and emoji
    screen.blit(frame_surface, (0, 0))
    display_emoji(screen, emotion_label)
    
    pygame.display.update()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            cap.release()
            pygame.quit()
            exit()

    # Exit on pressing 'q'
    keys = pygame.key.get_pressed()
    if keys[pygame.K_q]:
        break

cap.release()
cv2.destroyAllWindows()
pygame.quit()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━